In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
sc=SparkContext()
spark=SparkSession(sc)

# TF, IDF, TF-IDF

**TF**: refers to **Term Frequency** which is the frequency of a particular term in the document.Higher the TF,higher the importance of that term to the document.
<br>**IDF**: refers to **Inverse Document Frequrncy** is the frequency of the document that contai a specific term.If a term is present in all documents,then Document frequency is amximum which is 1 and the Inverse Doc frequency is minimum.The higher the IDF, the more relevant the term is.
<br>**TF-IDF**: refers to **Term Frequency-Inverse Document Frequency** is the product of TF and IDF.A high TF-IDF is obtained when both TF and IDF is high.(ie, frequency of term in one doc is high and frequency in all docs is low)

## Term Frequency, HashingTF, CountVectorizer

**HashingTF**: The HashingTF() utilizes the "*Murmurhash 3*" function to map a raw feature (a term) into an index (a number). Hashing is the process of transforming data of arbitrary size to size-fixed, usually shorter data. The term frequencies are calculated based on the generated indices. For the HashingTF() method, the mapping process is very cheap. Because each term-to-index mapping is independent of other term-to-index mapping. The hashing function takes a unique input and gerenate a “unique result”. However, **hashing collision** may occur, which means different features (terms) may be hased to the same index.

**CountVectorizer()**: It indexes terms by descending order of term frequencies in the entire corpus, NOT the term frequencies in the document. After the indexing process, the term frequencies are calculated by documents.

In [4]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
pdf = pd.DataFrame({
        'terms': [
            ['spark', 'spark', 'spark', 'is', 'awesome', 'awesome'],
            ['I', 'love', 'spark', 'very', 'very', 'much'],
            ['everyone', 'should', 'use', 'spark']
        ]
    })
df = spark.createDataFrame(pdf)
df.show(truncate=False)

+-------------------------------------------+
|terms                                      |
+-------------------------------------------+
|[spark, spark, spark, is, awesome, awesome]|
|[I, love, spark, very, very, much]         |
|[everyone, should, use, spark]             |
+-------------------------------------------+



### HashingTF

The **numFeatures** takes an integer,which should be larger than the total number of terms in the corpus and it should be power of 2.

In [6]:
from pyspark.ml.feature import HashingTF
from pyspark.ml.pipeline import Pipeline
hashtf=HashingTF(numFeatures=pow(2,4),inputCol='terms',outputCol='features(numFeatures), [index], [term frequency]')

stages=[hashtf]
pipeline=Pipeline(stages=stages)

In [7]:
pipeline.fit(df).transform(df).show(truncate=False)


+-------------------------------------------+------------------------------------------------+
|terms                                      |features(numFeatures), [index], [term frequency]|
+-------------------------------------------+------------------------------------------------+
|[spark, spark, spark, is, awesome, awesome]|(16,[1,15],[4.0,2.0])                           |
|[I, love, spark, very, very, much]         |(16,[0,1,2,8,12],[1.0,1.0,1.0,2.0,1.0])         |
|[everyone, should, use, spark]             |(16,[1,9,13],[2.0,1.0,1.0])                     |
+-------------------------------------------+------------------------------------------------+



Above,both terms 'spark' and 'is' is hashed 1 due to hashing collision.So,it has term frequency of 4.0 corresponding to three 'spark' terms and one 'is' term.The likelihood of hashing collision can be decreased by increasing the value of **numFeatures** parameter in *HashingTF* whose default value is pow(2,18)=262,144

### CountVectorizer()

The CountVectorizer() has three parameters to control which terms will be kept as features:
    <ul>
    <li>minTF: features that has term frequency less than minTF will be removed. If minTF=1, then no features will be removed.</li>
    <li>minDF: features that has document frequency less than minDF will be removed. If minDF=1, then no features will be removed.</li>
    <li>vocabSize: keep terms of the top vocabSize frequencies.</li>
    </ul>

In [8]:
from pyspark.ml.feature import CountVectorizer

countvec=CountVectorizer(minTF=1,minDF=1,vocabSize=20, inputCol='terms', outputCol='features(vocabSize),[index], [term frequency]')
stages=[countvec]
pipeline=Pipeline(stages=stages)

In [9]:
pipeline.fit(df).transform(df).show(truncate=False)

+-------------------------------------------+---------------------------------------------+
|terms                                      |features(vocabSize),[index], [term frequency]|
+-------------------------------------------+---------------------------------------------+
|[spark, spark, spark, is, awesome, awesome]|(10,[0,2,4],[3.0,2.0,1.0])                   |
|[I, love, spark, very, very, much]         |(10,[0,1,3,5,7],[1.0,2.0,1.0,1.0,1.0])       |
|[everyone, should, use, spark]             |(10,[0,6,8,9],[1.0,1.0,1.0,1.0])             |
+-------------------------------------------+---------------------------------------------+

